<a href="https://colab.research.google.com/github/ChristyMariaJoseph/palmpay/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from PIL import Image
from googleapiclient.http import MediaIoBaseUpload
from google.colab.patches import cv2_imshow
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
# Replace 'folder_id' with the actual ID of the Google Drive folder containing the images to be rotated
folder_id = '1h_Enb2qS306a_hcajeBHYFCRdKZ0VQiN'

In [ ]:
destination_folder_id = '1h_Enb2qS306a_hcajeBHYFCRdKZ0VQiN'

In [ ]:
# Authenticate with the Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

In [ ]:
# Define query parameters to retrieve files within the folder
query = f"'{folder_id}' in parents"  # Query to retrieve files within the folder
fields = "nextPageToken, files(id, name, mimeType)"  # Fields to include in the API response

In [ ]:
# Retrieve the list of files within the folder
results = drive_service.files().list(q=query, fields=fields).execute()
files = results.get('files', [])
fixed_resolution = (600,600)

In [ ]:
# Loop through the files in the folder
for file in files:
    file_id = file['id']
    filename = file['name']
    mimetype = file['mimeType']
    # Check if the file is an image
    if mimetype.startswith('image/'):
   # Download the image from Google Drive
     request = drive_service.files().get_media(fileId=file_id)
     image_bytes = io.BytesIO(request.execute())
     # Open the image using PIL (Pillow)
     image = Image.open(image_bytes)  
     import cv2
     # Convert the image to a numpy array
     image_array = np.array(image)
     gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
     kernel = np.ones((7, 7), np.float32) / 49
     blurred_image = cv2.filter2D(gray, -1, kernel) 
     noiseReduced = cv2.fastNlMeansDenoising(blurred_image)
     bot_1_kernal = np.ones((7,7),np.uint8)
     eqimg = cv2.morphologyEx(noiseReduced, cv2.MORPH_OPEN, bot_1_kernal)
     equ = cv2.equalizeHist(eqimg)
     inv = 255 - equ
     kernel = np.ones((7, 7), np.float32) / 49
     blurred_img = cv2.filter2D(inv, -1, kernel)
     img = blurred_img.copy()
     skel = img.copy()
     skel[:,:] = 0
     kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
     while cv2.countNonZero(img) > 0:
       eroded = cv2.morphologyEx(img, cv2.MORPH_ERODE, kernel)
       temp = cv2.morphologyEx(eroded, cv2.MORPH_DILATE, kernel)
       temp  = cv2.subtract(img, temp)
       skel = cv2.bitwise_or(skel, temp)
       img[:,:] = eroded[:,:]
     ret, thr1 = cv2.threshold(skel,2,255, cv2.THRESH_BINARY)
     thr1 = Image.fromarray(thr1)
     thr1_bytes = io.BytesIO()
     thr1.save(thr1_bytes, format=image.format)
     # Upload the rotated image back to Google Drive in the destination folder
     media = MediaIoBaseUpload(thr1_bytes, mimetype='image/jpeg', resumable=True)
     file_metadata = {'name': f'{filename}'}
     file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
      ).execute()
      # Move the rotated image to the destination folder
     file_id = file['id']
     destination_folder_id = '1rXezt47a3H8Kx6RVQyaBJgp61VcawKR0' # Replace with the correct destination folder ID
     file_metadata = {
      'addParents': destination_folder_id,
      'removeParents': file_id,
     }
     file = drive_service.files().update(
       fileId=file_id,
       body=file_metadata,
       fields='id, parents'
     ).execute()  